In [2]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
from datetime import date
import dateutil.parser
from dateutil.relativedelta import relativedelta
import unicodedata
from os.path import exists as file_exists
#To add wait time between requests
import time
import tweepy

In [3]:
# # your bearer token
MY_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAALgAbgEAAAAAx4tZoUWczbnuJ9NqinRBNNWrwac%3DvbWz10MQjanuN5wHE8putCUzkLkeoh8FpvOPYO2zYtRQWS3b2E"
# create your client 
client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

consumer_key= 'VXpzQlNVNzMtcURQeERzcEF2UFE6MTpjaQ'
consumer_secret= 'ayfPsW_L3fApjqvwbEz--lcsCHWvKQXao6dU2loDMpFox6IyF1'
access_token= '2971070979-x8hcqcy2A2AjTKTVwS5zA5ciSSYsBMW0NqoJFsc'
access_token_secret= '3CZNw4gK63S4a0tyjEgKZiaXRQpyoipa63SfNgSGkwolJ'

client = tweepy.Client(bearer_token=MY_BEARER_TOKEN,consumer_key= consumer_key,consumer_secret= consumer_secret,access_token= access_token,access_token_secret= access_token_secret,
                      )

In [4]:
search_query = "#covid19 -in:retweets"

In [5]:
def extract_info(t):
    metrics = t.public_metrics 
    
    result =[t.id,t.text,t.created_at.strftime("%m/%d/%Y %H:%M:%S"),
             metrics["retweet_count"],metrics["reply_count"],metrics["like_count"],
            metrics["quote_count"],t.attachments != None]
    result ={
        
        "tweet_id":t.id,
        "user":curr_username,
        "text":t.text,
        "date":t.created_at.strftime("%m/%d/%Y %H:%M:%S"),
        "retweets":metrics["retweet_count"],
        "replies":metrics["reply_count"],
        "likes":metrics["like_count"],
        "quotes":metrics["quote_count"],
        "has_viz":t.attachments != None}
    return result

In [7]:
usernames = ["DrEricDing","CDCGov","CNN","BNONews","ScottGottliebMD","DrTomFrieden","DNC"]
ids =client.get_users(usernames =usernames)

for j in range(len(usernames)):
    curr_id = ids.data[j].id
    curr_username = usernames[j]
    print(curr_id)
    start_date = date(2019,12,1)
    end_date = date(2019,12,31)
    tweet_date_format = "%Y-%m-%dT%H:%M:%SZ"
    for i in range(29):
        filename = f"data/{curr_username}_month{i+1}.csv"
        if file_exists(filename):
            start_date += relativedelta(months=1)
            end_date += relativedelta(months=1)
            continue;
        print(f"{curr_username} month {i+1} : {start_date.strftime(tweet_date_format)} to {end_date.strftime(tweet_date_format)} ")
        tweets = client.get_users_tweets(id=curr_id,
                                         start_time=start_date.strftime(tweet_date_format),
                                         end_time=end_date.strftime(tweet_date_format),
                                         #tweet_fields=["public_metrics"],
                                         tweet_fields = ["public_metrics","created_at", "text", "source","attachments","entities"],
                                         user_fields = ["name", "username", "location", "verified", "description"],
                                         max_results = 100,
                                         expansions=["author_id",'attachments.media_keys'],
                                         media_fields =["type","preview_image_url","public_metrics"]
                                         )
        #next month
        start_date += relativedelta(months=1)
        end_date += relativedelta(months=1)

        if tweets.data is None:
            continue
        data = list(map(lambda t:extract_info(t)  if str.__contains__(t.text.lower(),"covid") else None ,tweets.data))
        data_df = pd.DataFrame(list(filter(None, data)))
        data_df.to_csv(filename,index=False)
# 

18831926
DrEricDing month 1 : 2019-12-01T00:00:00Z to 2019-12-31T00:00:00Z 
DrEricDing month 2 : 2020-01-01T00:00:00Z to 2020-01-31T00:00:00Z 
DrEricDing month 3 : 2020-02-01T00:00:00Z to 2020-02-29T00:00:00Z 
DrEricDing month 4 : 2020-03-01T00:00:00Z to 2020-03-29T00:00:00Z 
DrEricDing month 5 : 2020-04-01T00:00:00Z to 2020-04-29T00:00:00Z 
DrEricDing month 6 : 2020-05-01T00:00:00Z to 2020-05-29T00:00:00Z 
DrEricDing month 7 : 2020-06-01T00:00:00Z to 2020-06-29T00:00:00Z 
DrEricDing month 8 : 2020-07-01T00:00:00Z to 2020-07-29T00:00:00Z 
DrEricDing month 9 : 2020-08-01T00:00:00Z to 2020-08-29T00:00:00Z 
DrEricDing month 10 : 2020-09-01T00:00:00Z to 2020-09-29T00:00:00Z 
DrEricDing month 11 : 2020-10-01T00:00:00Z to 2020-10-29T00:00:00Z 
DrEricDing month 12 : 2020-11-01T00:00:00Z to 2020-11-29T00:00:00Z 
DrEricDing month 13 : 2020-12-01T00:00:00Z to 2020-12-29T00:00:00Z 
DrEricDing month 14 : 2021-01-01T00:00:00Z to 2021-01-29T00:00:00Z 
DrEricDing month 15 : 2021-02-01T00:00:00Z to 20